# Apriori Principle

## [Dr Kieu's Lecture Notes on Apriori Principle](http://120.108.116.237/~ktduc/DA/Lecs/Topic04%20Association%20Analysis%20Basic%20Methods%20Jiawei%20Han.pdf)

## Python implementation from Lab (from scratch)

### Imports

In [3]:
##source: http://adataanalyst.com/machine-learning/apriori-algorithm-python-3-0/

import numpy as np

### Creating the `dataSet`

In [4]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])

    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we can use it as a key in a dict  

### Generating $L_1$ and $C_1$

In [5]:
#generates L1 from C1. Also returns a dictionary with support values.
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

### Testing

In [6]:
#----Testing-------
dataSet = loadDataSet()
print(dataSet)

C1 = createC1(dataSet)   # frozen sets individual items
print(C1)

#D is a dataset in the setform.
D = list(map(set,dataSet))

L1,suppDat0 = scanD(D,C1,0.5)
print(L1)
print(suppDat0) # the frequency in L1, the ones greater than min=0.5, notice that 4 is not in L1 because its frequncey is 0.25<5
#------------------

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]
{frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75}


### Generating $C_k$

In [7]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

### Generating all $C_k$'s
Eliminating infrequent itemsets

In [8]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

### Testing

In [9]:
#------Testing---------------
L,suppData = apriori(dataSet)
print("L:", L)    # All L's

# Individual L's
print("L0:", L[0])
print("L1:", L[1])
print("L2:", L[2])
print("L3:", L[3])  # no itemsets of size 4 exist

# Frequency data for each L
print(suppData)
#----------------------------

L: [[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})], [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})], [frozenset({2, 3, 5})], []]
L0: [frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]
L1: [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]
L2: [frozenset({2, 3, 5})]
L3: []
{frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75, frozenset({1, 3}): 0.5, frozenset({2, 5}): 0.75, frozenset({3, 5}): 0.5, frozenset({2, 3}): 0.5, frozenset({1, 5}): 0.25, frozenset({1, 2}): 0.25, frozenset({2, 3, 5}): 0.5}


### Generates Rules

In [ ]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

### Calculates the Confidence

In [ ]:
#calcConf() calculates the confidence of the rule and then find out the which rules meet the minimum confidence.
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

### Making rules based on Confidence Data

In [10]:
#rulesFromConseq() generates more association rules from our initial dataset. 
#This takes a frequent itemset and H, which is a list of items that could be on the right-hand side of a rule.
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)            

### Testing

In [11]:
#----Testing-----------
L,suppData= apriori(dataSet,minSupport=0.5)
rules= generateRules(L,suppData, minConf=0.7)
print(rules)
#----------------------

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0
[(frozenset({5}), frozenset({2}), 1.0), (frozenset({2}), frozenset({5}), 1.0), (frozenset({1}), frozenset({3}), 1.0)]
